In [3]:
import torch
import torch.nn as nn
from tqdm import tqdm

import config
import data
import preprocess

SyntaxError: invalid syntax (data.py, line 62)

In [ ]:
def get_padding(padding, kernel_size, dilation):
    kernel_size_type = type(kernel_size)
    if kernel_size_type != list and kernel_size_type != tuple:
        kernel_size = [kernel_size]

    padding_list = []
    for k in kernel_size:
        if padding == "same":
            p = ((k - 1)*dilation + 1)//2
        elif padding == "causal":
            p = (k - 1)*dilation
        elif padding == "valid":
            p = 0
        else:
            raise ValueError("Padding should be 'valid', 'same' or 'causal'")

        padding_list.append(p)

    if kernel_size_type != list and kernel_size_type != tuple:
        padding = padding_list[0]
    else:
        padding = padding_list

    return padding


class Conv1d(nn.Module):
  def __init__(self, input_size, output_size, kernel_size=1, stride=1, padding="same", dilation=1, bias=True):
    self.layer = nn.Conv1d(input_size, output_size, kernel_size=kernel_size, stride=stride, padding=get_padding(padding), dilation=dilation, bias=bias)

  def forward(self, x):
    length = x.size(-1)

    return self.layer(x)[..., :length]


class HighwayConv1d(nn.Module):
  def __init__(self, input_size, output_size, kernel_size=3, padding="same", dilation=1, dropout=0.0):
    super(HighwayConv1d, self).__init__()

    self.conv1d = Conv1d(input_size, 2*output_size, kernel_size=kernel_size, padding=padding, dilation=dilation)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    h1, h2 = self.conv1d(x).chunk(2, dim=1)
    h1 = torch.sigmoid(h1)
    h2 = torch.relu(h2)

    h = h1*h2 + (1.0 - h1)*x
    h = self.dropout(h)

    return h


class Model(nn.Module):
    def __init__(self, num_note, mel_size, hidden_size, dropout=0.0):
        self.embedding = nn.Embedding(num_note, hidden_size)
        self.encoder = nn.Sequential(
            Conv1d(num_note, hidden_size),
            nn.ReLU(),
            HighwayConv1d(hidden_size, hidden_size, dilation=1, dropout=dropout),
            HighwayConv1d(hidden_size, hidden_size, dilation=3, dropout=dropout),
            HighwayConv1d(hidden_size, hidden_size, dilation=9, dropout=dropout),
            HighwayConv1d(hidden_size, hidden_size, dilation=27, dropout=dropout))
    
        self.decoder = nn.Sequential(
            nn.Conv1d(hidden_size, hidden_size),
            nn.ReLU(),
            HighwayConv1d(hidden_size, hidden_size, padding="causal", dilation=1, dropout=dropout),
            HighwayConv1d(hidden_size, hidden_size, padding="causal", dilation=3, dropout=dropout),
            HighwayConv1d(hidden_size, hidden_size, padding="causal", dilation=9, dropout=dropout),
            HighwayConv1d(hidden_size, hidden_size, padding="causal", dilation=27, dropout=dropout),
            HighwayConv1d(hidden_size, hidden_size, padding="causal", dilation=1, dropout=dropout),
            HighwayConv1d(hidden_size, hidden_size, padding="causal", dilation=1, dropout=dropout),
            Conv1d(hidden_size, hidden_size),
            nn.ReLU(),
            Conv1d(hidden_size, mel_size))
    
    def forward(self, x):
        y = self.embedding(x['note'])
        y = self.encoder(y)
        y = self.decoder(y)

        return y

In [ ]:
# Run preprocess
preprocess.run()

NameError: name 'preprocess' is not defined

In [ ]:
dataloader = data.load()
model = Model().cuda()
optimizer = nn.Adam(model.parameters(), config.learning_rate)
criterion = nn.L1Loss()

for epoch in range(int(config.stop_step/config.save_step)):
    model.train()
    progress_bar = tqdm(range(config.save_step), leave=False)
    for _ in progress_bar:
        note, mel = next(dataloader['train'])
        note = note.cuda()
        mel = mel.cuda()
        
        optimizer.zero_grad()
        mel_gen = model(note)
        loss = criterion(mel_gen, mel)
        loss.backward()
        optimizer.step()
        progress_bar.set_description(f"Loss - {loss.item():.4f}")